<a href="https://colab.research.google.com/github/ccal2/dataScienceProject/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução


Esse projeto foi desenvolvido utilizando dois arquivos `.csv` que contêm dados relacionados aos livros da série *Game of Thrones* (A Guerra dos Tronos).

Os arquivos foram baixados diretamente de um dataset do [kaggle](https://www.kaggle.com/mylesoneill/game-of-thrones).


No arquivo `battles.csv` temos informações de várias batalhas que ocorreram durante a história e em `character-deaths.csv` podemos ver uma lista de personagens com dados relacionados à suas mortes.

# Setup

**Lembre-se de dar upload dos arquivos `battles.csv` e `character-deaths.csv`.**

In [1]:
import pandas as pd
import numpy as np

In [14]:
battles = pd.read_csv('battles.csv')
deaths = pd.read_csv('character-deaths.csv')